<a href="https://colab.research.google.com/github/spe301/AI-generated-AI/blob/main/AI_generated_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install Potosnail==0.0.6

# Text Classification

In [2]:
import pandas as pd
import numpy as np
from google.colab import files
import zipfile
import io
from potosnail import MachineLearning, DeepLearning, DataHelper, Evaluater, Algorithms, Wrappers

ml = MachineLearning()
dl = DeepLearning()
dh = DataHelper()
ev = Evaluater()
al = Algorithms()
wr = Wrappers()

df = pd.read_csv('https://raw.githubusercontent.com/spe301/Wikipedia-Capstone/main/Data/HumanText.csv').drop(['Unnamed: 0', 'Unnamed: 0.1'], axis='columns')
df2 = pd.read_csv('https://raw.githubusercontent.com/spe301/Wikipedia-Capstone/main/Data/Generated%20(1).csv').drop(['Unnamed: 0'], axis='columns')
blog_df = pd.concat([df.sample(n=180), df2])

df = pd.read_csv('https://raw.githubusercontent.com/spe301/dsc-phase-4-project/main/data/judge-1377884607_tweet_product_company.csv', engine='python',encoding='latin-1')
df = df.drop(6)
df = df.loc[df['is_there_an_emotion_directed_at_a_brand_or_product'] != "I can't tell"]
ix = list(df.index)
clean = []
for i in range(len(ix)):
  clean.append(df['tweet_text'][ix[i]].replace('.@', '').replace('.', '').replace('!', '').replace('@', '').replace('?', '').replace(':', '')
  .replace(',', '').replace(';', '').lower())

df['tweet'] = clean
tweet_df = df.drop(['tweet_text', 'emotion_in_tweet_is_directed_at'], axis='columns')

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
def Preprocess(df, data_str, target_str):
  '''fully preprocesses a dataframe of Text Documents for classification'''
  avg = 0
  for i in range(len(df[data_str])):
    avg += len(list(df[data_str])[i].split(' '))/len(df)
  pad = int(avg)
  text, labels = dl.ModelReadyText1(df[data_str], df[target_str], pad)
  return text, labels

blogs, blog_labels = Preprocess(blog_df, 'Text', 'AI')
tweets, tweet_labels = Preprocess(tweet_df, 'tweet', 'is_there_an_emotion_directed_at_a_brand_or_product')

In [6]:
def GetVocab(df, data_str):
  words = []
  for i in range(len(df)):
    word_lst = list(df[data_str])[i].replace('\n', ' ').split(' ')
    for word in word_lst:
      words.append(word.replace('.', '').replace(',', '').replace(' ', '').replace('"', '').replace(':', '').replace(';', '').replace('!', ''))
  return len(np.unique(words))

In [8]:
blogs_vocab = GetVocab(blog_df, 'Text')
tweets_vocab = GetVocab(tweet_df, 'tweet')

In [26]:
tweets.shape

(8936, 17)

In [28]:
blogs_grid = {'output_dim': [tweet_labels.shape[1]], 'embedding': [tweets_vocab], 'nodes': [64], 'activation': ['tanh'], 
              'regularizer': [None, 'L1'], 'stacking': [False, True], 'dropout': [False], 'optimizer': ['adam'], 
              'method': ['GRU'], 'bidirectional': [True]} 

dl.CollectPerformance(blogs_grid, dl.RNN, tweets, tweet_labels.reshape(-1))

preforming 4 total fits ...
Epoch 1/50
224/224 [==============================] - 11s 33ms/step - loss: 0.7120 - accuracy: 0.6447 - val_loss: 0.6374 - val_accuracy: 0.6667
Epoch 2/50
224/224 [==============================] - 7s 29ms/step - loss: 0.6367 - accuracy: 0.6679 - val_loss: 0.6447 - val_accuracy: 0.6655
Epoch 3/50
224/224 [==============================] - 7s 30ms/step - loss: 0.5746 - accuracy: 0.7098 - val_loss: 0.6944 - val_accuracy: 0.6040
Epoch 4/50
224/224 [==============================] - 6s 29ms/step - loss: 0.4142 - accuracy: 0.8150 - val_loss: 0.8133 - val_accuracy: 0.6068
Epoch 5/50
224/224 [==============================] - 6s 29ms/step - loss: 0.3252 - accuracy: 0.8461 - val_loss: 1.0198 - val_accuracy: 0.5615
Epoch 6/50
224/224 [==============================] - 6s 29ms/step - loss: 0.2669 - accuracy: 0.8627 - val_loss: 1.3212 - val_accuracy: 0.5319
1 of 4 fits complete!
Epoch 1/50
224/224 [==============================] - 11s 31ms/step - loss: 0.7239 - accura

,output_dim,embedding,nodes,activation,regularizer,stacking,dropout,optimizer,bidirectional,epochs,batch_size,method,accuracy,loss,val_accuracy,val_loss
0,3,12300,64,tanh,None,False,False,adam,True,6,32,GRU,0.858422,0.274318,0.531879,1.321163
1,3,12300,64,tanh,None,True,False,adam,True,6,32,GRU,0.848209,0.304630,0.601790,0.912252
2,3,12300,64,tanh,L1,False,False,adam,True,6,32,GRU,0.851427,0.280485,0.588926,1.135162
3,3,12300,64,tanh,L1,True,False,adam,True,6,32,GRU,0.841074,0.322878,0.592841,0.866836


# Image Classification

In [29]:
from google.colab import files
import zipfile
import io
uploaded = files.upload()
data = zipfile.ZipFile(io.BytesIO(uploaded['IMGDS1.zip']), 'r')
data.extractall()

Saving IMGDS1.zip to IMGDS1.zip


In [31]:
train_dir = r'dataset/training_set'
test_dir = r'dataset/test_set'
train_dogs = r'dataset/training_set/dogs'
train_cats = r'dataset/training_set/cats'
test_dogs = r'dataset/test_set/dogs'
test_cats = r'dataset/test_set/cats'
directories = [train_dir, test_dir, train_dogs, train_cats, test_dogs, test_cats]
classes = ['cats', 'dogs']

tri1, tei1, tray1, tey1 = dl.ModelReadyPixles(directories, classes)

Found 8000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.
Found 8000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.
Found 8000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.
Found 8000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [35]:
cnn_grid = {'output_dim': [tray1.shape[1]], 'base_filters':[32], 'kernel_size': [3], 'activation': ['relu'], 
            'nblocks': [3, 4], 'pool':[2], 'dropout': [False], 'closer': [False, True], 'optimizer': ['adam'], 
            'metrics': ['accuracy']}

dl.CollectPerformance(cnn_grid, dl.CNN, tri1, tray1, epochs=3)

preforming 4 total fits ...
Epoch 1/3
200/200 [==============================] - 227s 1s/step - loss: 0.6961 - accuracy: 0.5115 - val_loss: 0.7131 - val_accuracy: 0.4894
Epoch 2/3
200/200 [==============================] - 220s 1s/step - loss: 0.6758 - accuracy: 0.5926 - val_loss: 0.6266 - val_accuracy: 0.6419
Epoch 3/3
200/200 [==============================] - 219s 1s/step - loss: 0.6200 - accuracy: 0.6594 - val_loss: 0.6312 - val_accuracy: 0.6169
1 of 4 fits complete!
Epoch 1/3
200/200 [==============================] - 220s 1s/step - loss: 0.6776 - accuracy: 0.5509 - val_loss: 0.5842 - val_accuracy: 0.6913
Epoch 2/3
200/200 [==============================] - 219s 1s/step - loss: 0.5624 - accuracy: 0.7050 - val_loss: 0.5013 - val_accuracy: 0.7581
Epoch 3/3
200/200 [==============================] - 219s 1s/step - loss: 0.4707 - accuracy: 0.7793 - val_loss: 0.4636 - val_accuracy: 0.7844
2 of 4 fits complete!
Epoch 1/3
200/200 [==============================] - 222s 1s/step - loss: 0.

,output_dim,base_filters,kernel_size,activation,nblocks,pool,dropout,closer,metrics,epochs,batch_size,optimizer,accuracy,loss,val_accuracy,val_loss
0,2,32,3,relu,3,2,False,False,accuracy,3,32,adam,0.663906,0.615597,0.616875,0.631234
1,2,32,3,relu,3,2,False,True,accuracy,3,32,adam,0.774844,0.471077,0.784375,0.463609
2,2,32,3,relu,4,2,False,False,accuracy,3,32,adam,0.731094,0.533612,0.737500,0.524788
3,2,32,3,relu,4,2,False,True,accuracy,3,32,adam,0.726250,0.539328,0.737500,0.509467


# Tabular Classification

# Tabular Regression

# Dogs and Cats
train_dir = r'dataset/training_set'

test_dir = r'dataset/test_set'

train_dogs = r'dataset/training_set/dogs'

train_cats = r'dataset/training_set/cats'

test_dogs = r'dataset/test_set/dogs'

test_cats = r'dataset/test_set/cats'

directories = [train_dir, test_dir, train_dogs, train_cats, test_dogs, test_cats]

classes = ['cats', 'dogs']